# <center>Cleaning Data</center>
### <center>by Mahdi Shahrabi</center>
In this report I will describe how to clean shareholder data crawled from TSE, and merge it with price data and the data of volumne of trades done by Individula/istitutional data.

## Loading and Modifying Shareholders Data

We use this libraries to manipulate data:

In [1]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import glob

# Printing All Results
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Since we have Farsi/Arabaci strings in our data, we use this function to convert all arabic characters to farsi characters. We run this function on all Farsi/Arabaci strings.

In [2]:
## Changing Arbic Characters to Persian Characters !
from convert_ar_characters import convert_ar_characters

First, we load data of daily shareholders from the folder it is saved in. In Nov 2019 the data is stored in about 110 chunks. However in each there are duplicated roww which we drop when we load each chunk.

In [5]:
%%time
## Loading Data of Firm Holder (Chunks)
os.chdir(r"D:\Shareholder Data\Sahreholders async html parser")
# os.chdir(r"D:\Shareholder Data\Sahreholders async html parser\Sample")

# Creating the list of files in the workspace and loading them
all_filenames = [i for i in glob.glob('*.csv')]
# # For 1399 Data
# os.chdir(r"D:\Shareholder Data\Sahreholders async html parser\1399")
# all_filenames = [i for i in glob.glob('*1399.csv')]

DataOrg = pd.concat([pd.read_csv(f,index_col=0) for f in all_filenames ])
lng = len(DataOrg)
DataOrg = DataOrg.drop_duplicates()

os.chdir(r"C:\Users\Mahdi\Dr. Heidari - TSE\WorkInHands")
print('The number of observations after dropping duplicates is: ',len(DataOrg),' and befor was about ',lng )


The number of observations after dropping duplicates is:  7185010  and befor was about  7385391
Wall time: 2min 17s


We, also, rename columns to names that we use frequently:

In [6]:
# Renaming Columns
DataOrg = DataOrg.rename(columns={'date':'Date_gre','group_name':'Industry','holder':'Shareholder_raw',
                                  'Holder':'Shareholder_raw','Name':'Symbol','Percent':'percent','Number':'quantity',
                                  'name':'Symbol','stock_id':'Id_tse','title':'firm_name'})

We convert strings in DataOrg to farsi characters. Since, applying function in each cell is time-cosuming we use dictionary and mapping.

In [7]:
%%time
# DataOrg.firm_GPname: convert_ar_characters(x)
Names = DataOrg.Industry.drop_duplicates()
Conv_Names = Names.apply(lambda x : convert_ar_characters(x))
DataOrg_Industry_ArtoFa_dict = dict(zip(Names,Conv_Names))
DataOrg['Industry'] = DataOrg.Industry.map(DataOrg_Industry_ArtoFa_dict)
# # For 1399 Data
# DataOrg['Industry'] = np.nan

# DataOrg.Shareholder_raw: convert_ar_characters(x)
Names = DataOrg.Shareholder_raw.drop_duplicates()
Conv_Names = Names.apply(lambda x : convert_ar_characters(x))
DataOrg_Shareholder_raw_ArtoFa_dict = dict(zip(Names,Conv_Names))
DataOrg['Shareholder_raw'] = DataOrg.Shareholder_raw.map(DataOrg_Shareholder_raw_ArtoFa_dict)

# DataOrg.Symbol: convert_ar_characters(x)
Names = DataOrg.Symbol.drop_duplicates()
Conv_Names = Names.apply(lambda x : convert_ar_characters(x))
DataOrg_Symbol_ArtoFa_dict = dict(zip(Names,Conv_Names))
DataOrg['Symbol'] = DataOrg.Symbol.map(DataOrg_Symbol_ArtoFa_dict)

# DataOrg.firm_name: convert_ar_characters(x)
Names = DataOrg.firm_name.drop_duplicates()
Conv_Names = Names.apply(lambda x : convert_ar_characters(x))
DataOrg_firm_name_ArtoFa_dict = dict(zip(Names,Conv_Names))
DataOrg['firm_name'] = DataOrg.firm_name.map(DataOrg_firm_name_ArtoFa_dict)
# # For 1399 Data
# DataOrg['firm_name'] = np.nan

Wall time: 11 s


We, also, creat Gregorian and Jalali date columns, along columns for year, month, and day.

In [8]:
%%time
## Modifying Date
DataOrg['True_Date'] = pd.to_datetime(DataOrg['Date_gre'], format='%Y%m%d')
G = DataOrg.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
DataOrg_date_GtoJ_dict = dict(zip(G,J))
DataOrg['Jalali_Date']=DataOrg.True_Date.map(DataOrg_date_GtoJ_dict)
DataOrg['year'] = DataOrg.Jalali_Date.apply(lambda x: x.year)
DataOrg['month'] = DataOrg.Jalali_Date.apply(lambda x: x.month)
DataOrg['day'] = DataOrg.Jalali_Date.apply(lambda x: x.day)

Wall time: 33 s


Next, we load data of unique name of shareholder 'Sh_spec_handy.xlsx' and convert all Arabic characters to Farsi characters to find unique name of shareholders.

In [9]:
%%time
# Setting current workspace to folder of Other Data
os.chdir(r"C:\Users\Mahdi\Dr. Heidari - TSE\WorkInHands\_Data\OtherData")
Sh_spec_handy=pd.read_excel("Sh_spec_handy.xlsx")
Sh_spec_handy.Shareholder_raw=Sh_spec_handy.Shareholder_raw.apply(lambda x: convert_ar_characters(x))
Sh_spec_handy.Shareholder=Sh_spec_handy.Shareholder.apply(lambda x: convert_ar_characters(x))
os.chdir(r"C:\Users\Mahdi\Dr. Heidari - TSE\WorkInHands")

Wall time: 5.07 s


Since not all shareholders name in our data has a name in 'Sh_spec_handy', we find new names and append it to 'Sh_spec_handy' and save it to "NewSh_spec_handy.xlsx". Then, we must complete this new list by hand, and use it for finding unique names in our shareholder data.

In [10]:
%%time
NewShH_raw=pd.DataFrame({"Shareholder_raw":list(set(DataOrg.Shareholder_raw.drop_duplicates())-set(Sh_spec_handy.Shareholder_raw))})
# Adding to the List We Had Before and Saving it in an Excel File
NewSh_spec_handy=Sh_spec_handy.append(NewShH_raw)
NewSh_spec_handy.to_excel("NewSh_spec_handy.xlsx")
print('Number of New Shareholder is: ',len(NewShH_raw))

ModuleNotFoundError: No module named 'openpyxl'

Using this file, we creat a dictionary and find Shareholder unique names:

In [11]:
%%time
# Creating a Dictionary
dict_ShH_raw_to_ShH = dict(zip(NewSh_spec_handy['Shareholder_raw'],NewSh_spec_handy['Shareholder']))
dict_ShH_raw_to_Typ = dict(zip(NewSh_spec_handy['Shareholder_raw'],NewSh_spec_handy['Type']))
# Maping
DataOrg['ShareHolder'] = DataOrg['Shareholder_raw'].map(dict_ShH_raw_to_ShH)
DataOrg['ShareHolder_Type'] = DataOrg['Shareholder_raw'].map(dict_ShH_raw_to_Typ)

Wall time: 4.62 s


In [12]:
%%time
# Meging shareholder with same sharehodler name but different sharehodler_raw name
lng = len(DataOrg)
DataOrg = DataOrg.groupby(['Symbol','Jalali_Date','ShareHolder'],as_index=False).agg({'Id_tse':'first',
                                                          'Industry':'first', 'percent':'sum',
                                                          'quantity':'sum','Shareholder_raw':'count',  
                                                          'ShareHolder_Type':'first','True_Date':'first', 'year':'first',
                                                          'month':'first', 'day':'first'})
print('Before merging size is ', lng, 'after merging size is ', len(DataOrg))

Before merging size is  7185010 after merging size is  5129548
Wall time: 6min 36s


To find firm industry/Industry_Id, we use file "name_groupName_groupId.xlsx" which is as follow:

After converting arabic characters and renaming, we create a dictionary and map each of the industries to Industry_Id for DataOrg

In [13]:
%%time
os.chdir(r"C:\Users\Mahdi\Dr. Heidari - TSE\WorkInHands\_Data\OtherData")
firm_indus = pd.read_excel("name_groupName_groupId.xlsx",index_col=0).drop(columns=['ID'])
os.chdir(r"C:\Users\Mahdi\Dr. Heidari - TSE\WorkInHands")
firm_indus.head()

## Maping Shareholder_raw to Symbol and Mapping 
firm_indus['Name'] = firm_indus.Name.apply(lambda x: convert_ar_characters(x))
firm_indus['Gorup_name'] = firm_indus.Gorup_name.apply(lambda x: convert_ar_characters(x))
firm_indus = firm_indus.rename(columns = {'Name':'Symbol','Gorup_name':'Industry','Group_id':'Industry_Id'})
dict_Industry_Id = dict(zip(firm_indus.Industry.drop_duplicates(),firm_indus.Industry_Id.drop_duplicates()))
# # For 1399 Data
# DataOrg["Industry"]=DataOrg.Symbol.map(dict(zip(firm_indus.Symbol,firm_indus.Industry)))
DataOrg["Industry_Id"]=DataOrg.Industry.map(dict_Industry_Id)
DataOrg["Fill_Flag"]=False

Wall time: 1.84 s


It is seen that for some firms, in some days, the sum of percentage of its shareholders exceeds 100%! We find these firm-days and drop them. Later, we substitute them with last observed shareholders-percentage.

In [14]:
%%time
## Finding and Dropping rows of DataOrg which sum of percentage is larger that 100% !!
temp = DataOrg.groupby(['Id_tse','Jalali_Date'],as_index=False).agg({'percent':'sum'})
out_over100 = temp[temp.percent>100]

# Printing output and sorting it
out_over100.sort_values(by=['Id_tse','Jalali_Date']).head()
print('number of observations with percent sum over 100 is: ',len(out_over100))

# droping observations with sum over 100 (Time-Consuming, I should find a better way)
lng = len(DataOrg)
DataOrg = DataOrg[~pd.Series(list(zip(DataOrg['Id_tse'], DataOrg['Jalali_Date']))).isin([(x,y) for (x,y) in zip(out_over100.Id_tse,out_over100.Jalali_Date)])]

print('number of observations before removing rows with percent-sum over 100 is: ',lng,'  and after removing rows with percent-sum over 100 is: ',len(DataOrg))

number of observations with percent sum over 100 is:  176
number of observations before removing rows with percent-sum over 100 is:  5129548   and after removing rows with percent-sum over 100 is:  5128083
Wall time: 6min 26s


## Loading other Data to Merge with shareholder Data

First, we load data from file "adjPrices_1399-01-09.csv" which is as follow:

In [15]:
%%time
os.chdir(r"D:\Shareholder Data\Price\Adj price")
Adj_Price=pd.read_csv("adjPrices_1399-01-09.csv",low_memory=False,index_col=0)

os.chdir(r"D:\Shareholder Data\Price\Unadj price")
UnAdj_Price=pd.read_csv("Stocks_Prices_1399-02-06.csv",low_memory=False,index_col=0)

os.chdir(r"C:\Users\Mahdi\Dr. Heidari - TSE\WorkInHands")
Adj_Price.head()

C:\Users\Mahdi\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Wall time: 7.52 s


,Date_gre,High,Low,Open,Last,Volume,close,ID
0,20150315,1102,1102,1102,1102,17995000,1102,22941065011246116
1,20150316,1157,1157,1157,1157,524171,1157,22941065011246116
2,20150317,1215,1215,1215,1215,572258,1215,22941065011246116
3,20150318,1276,1276,1276,1276,200322,1276,22941065011246116
4,20150325,1339,1339,1339,1339,185325,1339,22941065011246116


Then, we create Gregorian and Jalali date for adjusted and unadjusted price data:

In [16]:
# Creating Date
Adj_Price['True_Date'] = pd.to_datetime(Adj_Price['Date_gre'], format='%Y%m%d')
G = Adj_Price.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
Adj_Price_date_GtoJ_dict = dict(zip(G,J))
Adj_Price['Jalali_Date']=Adj_Price.True_Date.map(Adj_Price_date_GtoJ_dict)
## Preparing for Filtering by Date
Adj_Price['year']=Adj_Price.Jalali_Date.apply(lambda x: x.year)
Adj_Price['month']=Adj_Price.Jalali_Date.apply(lambda x: x.month)


# Creating Date
UnAdj_Price['True_Date'] = pd.to_datetime(UnAdj_Price['date'], format='%Y%m%d')
G = UnAdj_Price.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
UnAdj_Price_date_GtoJ_dict = dict(zip(G,J))
UnAdj_Price['Jalali_Date']=UnAdj_Price.True_Date.map(UnAdj_Price_date_GtoJ_dict)
## Preparing for Filtering by Date
UnAdj_Price['year']=UnAdj_Price.Jalali_Date.apply(lambda x: x.year)
UnAdj_Price['month']=UnAdj_Price.Jalali_Date.apply(lambda x: x.month)

In [17]:
%%time
# Merging Different kind of prices
PRICE = pd.merge(Adj_Price,UnAdj_Price[['stock_id','Jalali_Date','close_price']],left_on=['ID','Jalali_Date'],right_on=['stock_id','Jalali_Date'],how='outer').rename(columns={'close_price':'Unadjusted_close'})
PRICE.rename(columns={'ID':'Id_tse'},inplace=True)

Wall time: 1min 11s


In addition, we load data of market index from file "Index", to hava a list of all woking dates.

In [18]:
os.chdir(r"D:\Shareholder Data\Index")
Index=pd.read_csv("indexes_1399-01-12.csv",low_memory=False,index_col=0)
os.chdir(r"C:\Users\Mahdi\Dr. Heidari - TSE\WorkInHands")

We choose usefull columns, create date, rename columns, reorder columns, and add a column "Fill_Flag" which, if ture, shows that if the data is filled.

In [19]:
# Choosing two needed columns
Index = Index[Index.index_id=='overall_index'].reset_index(drop=True)

# Creating Date Column

Index['Jalali_Date']=Index.date.apply(lambda x: jd.date(year=int(x[0:x.find('/')]),
                                                        month=int(x[(x.find('/')+1):x.rfind('/')]),
                                                        day=int(x[(x.rfind('/')+1):])))

Index['year'] = Index.Jalali_Date.apply(lambda x: x.year)
Index['month'] = Index.Jalali_Date.apply(lambda x: x.month)
Index['day'] = Index.Jalali_Date.apply(lambda x: x.day)

Index['True_Date']=Index.Jalali_Date.apply(lambda x: jd.date.togregorian(x))



# Renaming Column
Index = Index.rename(columns={'index':'close'})
# Reordering Columns
Index = Index[['Jalali_Date','close','True_Date','year','month']]
# Flag for being filled
Index['Fill_Flag']=False

For our work we must filter data by date using these lines of codes. However, we do not filter them yet.

In [20]:
%%time
## Filtering Data by Date
start_year = 1390
start_month = 1
end_year = 1398
end_month = 12
# Selecting Data From 1390, Farvardin and 1398 Esfand
print('The number of observations before filtering by date is: ',len(DataOrg))
DataOrg = DataOrg[np.logical_or(DataOrg.year>start_year,
                                np.logical_and(DataOrg.year==start_year,DataOrg.month>=start_month))]
DataOrg = DataOrg[np.logical_or(DataOrg.year<end_year,
                                np.logical_and(DataOrg.year==end_year,DataOrg.month<=end_month))]
print('The number of observations after filtering by date is: ',len(DataOrg))

# Selecting Data From 1390, Farvardin and 1398 Sharivar
Adj_Price = Adj_Price[np.logical_or(Adj_Price.year>start_year,
                                    np.logical_and(Adj_Price.year==start_year,Adj_Price.month>=start_month))]
Adj_Price = Adj_Price[np.logical_or(Adj_Price.year<end_year,
                                    np.logical_and(Adj_Price.year==end_year,Adj_Price.month<=end_month))]

# Selecting Data From 1390, Farvardin and 1398 Sharivar
UnAdj_Price = UnAdj_Price[np.logical_or(UnAdj_Price.year>start_year,
                                    np.logical_and(UnAdj_Price.year==start_year,UnAdj_Price.month>=start_month))]
UnAdj_Price = UnAdj_Price[np.logical_or(UnAdj_Price.year<end_year,
                                    np.logical_and(UnAdj_Price.year==end_year,UnAdj_Price.month<=end_month))]

# Selecting Data From 1390, Farvardin and 1398 Sharivar
Index = Index[np.logical_or(Index.year>start_year,
                            np.logical_and(Index.year==start_year,Index.month>=start_month))]
Index = Index[np.logical_or(Index.year<end_year,
                            np.logical_and(Index.year==end_year,Index.month<=end_month))]

The number of observations before filtering by date is:  5128083
The number of observations after filtering by date is:  4394783
Wall time: 4.56 s


## Filling Gaps in Data

For Ids in shareholder-daily data we fill all missed dates between dates that we have data for. We make its 'Fill_Flag' True. (Based on the 'stock' not the 'holder')

In [21]:
%%time
## Filling Holder Information
# Initilizing the dataframe of filled data
DataOrg_filter_stock_missed = pd.DataFrame(columns=DataOrg.columns.to_list())

cnt = 0
Ids_DataOrg = DataOrg.Id_tse.drop_duplicates()
lng = len(Ids_DataOrg)

for Id in Ids_DataOrg:
    if cnt%100==0:
        print('Id = ',Id,': ',cnt,' from ',lng)
    # Selecting Data for stock x and setting fill_flag false again
    data_sym = DataOrg[DataOrg.Id_tse==Id]
    data_sym['Fill_Flag'] = False
    # Initilizing Missed_Data for stock x
    Missed_Data = pd.DataFrame(columns=data_sym.columns.to_list())
    # Extracting dates availabe in holder information
    Org_sym_Dates = pd.DataFrame(data_sym.Jalali_Date.drop_duplicates().sort_values(),columns=['Jalali_Date'])
    # Extracting all working days
    sym_dates_index=Index[np.logical_and(Index.Jalali_Date>=
        Org_sym_Dates.Jalali_Date.iloc[0],Index.Jalali_Date<=Org_sym_Dates.Jalali_Date.iloc[-1])].reset_index(drop=True)
    sym_dates_index = sym_dates_index.sort_values(by=['Jalali_Date'],ascending=True)
    # Finding missed dates and sorting them
    Missed_Dates=list(pd.DataFrame(set(sym_dates_index.Jalali_Date)-
                    set(Org_sym_Dates.Jalali_Date),columns=['Jalali_Date']).sort_values(by='Jalali_Date').Jalali_Date)

    # for all missed dates
    for missdate in Missed_Dates:
        # finding index of missdate in the list of woriking days
        l = sym_dates_index[sym_dates_index.Jalali_Date==missdate].index.to_list()
        # Initilizing the missed data for this day for stock x
        Missed_Data_temp = pd.DataFrame(columns=data_sym.columns.to_list())
        # choosing the data for fillingn missed data
        data_for_filling = data_sym[data_sym.Jalali_Date == sym_dates_index.Jalali_Date.iloc[l[0]-1]]
        # filling missed data
        Missed_Data_temp[['Industry','Industry_Id', 'Shareholder_raw', 'Symbol', 'percent','quantity', 'Id_tse',
                          'ShareHolder', 'ShareHolder_Type']] = data_for_filling[['Industry','Industry_Id', 'Shareholder_raw',
                             'Symbol', 'percent','quantity', 'Id_tse', 'ShareHolder', 'ShareHolder_Type']]
        # filling missed data - dates
        Missed_Data_temp.Fill_Flag = True
        Missed_Data_temp.Jalali_Date = missdate
        Missed_Data_temp.True_Date = Missed_Data_temp.Jalali_Date.apply(lambda x: jd.date.togregorian(x))
        Missed_Data_temp.Date_gre  = Missed_Data_temp.True_Date.apply(lambda x: x.year*10000+x.month*100+x.day)
        Missed_Data_temp.year = Missed_Data_temp.Jalali_Date.apply(lambda x: x.year)
        Missed_Data_temp.month  = Missed_Data_temp.Jalali_Date.apply(lambda x: x.month)
        Missed_Data_temp.day  = Missed_Data_temp.Jalali_Date.apply(lambda x: x.day)
        
        # appending missed data for a day to missed data for stock x
        Missed_Data = Missed_Data.append(Missed_Data_temp)
    
    # appending missed data for stock x to all missed data
    DataOrg_filter_stock_missed = DataOrg_filter_stock_missed.append(Missed_Data)
    cnt+=1

# appending all missed data to data we have after filltering by stocks
print('Number of filled rows in holder data is: ',len(DataOrg_filter_stock_missed))
DATAORG = DataOrg.append(DataOrg_filter_stock_missed.drop_duplicates()).sort_values(by=['Symbol','Jalali_Date'])

# List of Missing Days Symbols
MissingDaySymbol = DATAORG[DATAORG.Fill_Flag==True].drop_duplicates(['Symbol','Jalali_Date']).sort_values(by=['Id_tse','Jalali_Date'])[['Jalali_Date','Id_tse','Symbol']]
MissingDaySymbol.to_csv("MissingDaySymbol_List.csv",index=False)
MissingDaySymbol.head()

Id =  34973883374080119 :  0  from  729


C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


Id =  29758477602878557 :  100  from  729
Id =  66726992874614788 :  200  from  729
Id =  27668158733246204 :  300  from  729
Id =  50652985928800943 :  400  from  729
Id =  19040514831923530 :  500  from  729
Id =  59486059679335017 :  600  from  729
Id =  50341528161302545 :  700  from  729
Number of filled rows in holder data is:  101966
Wall time: 1h 28min 31s
Parser   : 433 ms


,Jalali_Date,Id_tse,Symbol
2921792,1392-11-07,408934423224097,فرآور
2923306,1393-12-27,408934423224097,فرآور
2925400,1395-11-06,408934423224097,فرآور
2925454,1395-12-03,408934423224097,فرآور
2925937,1396-08-07,408934423224097,فرآور


## Merging Data

In [22]:
# # For 1399 Data
# DATAORG = DataOrg.sort_values(by=['Symbol','Jalali_Date'])

In [23]:
%%time
os.chdir(r"D:\Shareholder Data\NumberOfShares")
# Creating the list of files in the workspace and loading them
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]
NumberOfShares = pd.concat([pd.read_csv(f,index_col=0).drop_duplicates() for f in all_filenames ])

# Creating Jalali Date
NumberOfShares['True_Date'] = pd.to_datetime(NumberOfShares['date'], format='%Y%m%d')
G = NumberOfShares.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
NumberOfShares_date_GtoJ_dict = dict(zip(G,J))
NumberOfShares['Jalali_Date']=NumberOfShares.True_Date.map(NumberOfShares_date_GtoJ_dict)

Names = NumberOfShares.name.drop_duplicates()
Conv_Names = Names.apply(lambda x : convert_ar_characters(x))
NumberOfShares_name_ArtoFa_dict = dict(zip(Names,Conv_Names))
NumberOfShares['name'] = NumberOfShares.name.map(NumberOfShares_name_ArtoFa_dict)

NumberOfShares = NumberOfShares.rename(columns={'name':'Symbol'})

NumberOfShares.head()

# Merging
DATAORG = pd.merge(DATAORG,NumberOfShares[['Shares','Symbol','Jalali_Date']],
                  how='left',left_on=['Jalali_Date','Symbol'],right_on=['Jalali_Date','Symbol'])

Wall time: 5min 31s


In [24]:
%%time
DATAORG = pd.merge(DATAORG,PRICE[['Id_tse','Jalali_Date','High','Low','Open','Last','Volume','close','Unadjusted_close']],how='left',left_on=['Jalali_Date','Id_tse',],right_on=['Jalali_Date','Id_tse'])
DATAORG['MarketCap'] = DATAORG['Shares']*DATAORG['Unadjusted_close']

Wall time: 6min 53s
Compiler : 406 ms


In [25]:
# Selecting Columns
DATAORG = DATAORG[['Id_tse','Symbol', 'Jalali_Date','Industry', 'ShareHolder', 'percent',
       'quantity', 'ShareHolder_Type',  'Shares','MarketCap', 'High', 'Low',
       'Open', 'Last', 'Volume', 'close', 'Unadjusted_close', 'Industry_Id', 'True_Date', 'year',
       'month', 'day','Fill_Flag']]

## Saving Data

In [26]:
# # For 1399 Data 
# Symbols = pd.DataFrame(DATAORG.Symbol.drop_duplicates(),columns=['Symbol'])
# Symbols['chnk_id'] = 1399
# dict__ = dict(zip(Symbols.Symbol,Symbols.chnk_id))
# DATAORG['chnk_id'] = DATAORG.Symbol.map(dict__)

In [27]:
%%time
Symbols = pd.DataFrame(DATAORG.Symbol.drop_duplicates(),columns=['Symbol'])
Symbols['chnk_id'] =0
cnt=0
for i in range(0, len(Symbols), 5):
    Symbols['chnk_id'][i:i+5] = cnt
    cnt+=1
dict__ = dict(zip(Symbols.Symbol,Symbols.chnk_id))
DATAORG['chnk_id'] = DATAORG.Symbol.map(dict__)


C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Wall time: 2.17 s


In [29]:
%%time
os.chdir(r"D:\Shareholder Data\Cleaned - Merged with Price")
gb = DATAORG.groupby(['chnk_id'])
lng = len(gb)
cnt=1
for name, group in gb:
    if cnt%5==1:
        print(cnt,' from ',lng)
    file_name = 'CHUNK_'+str(name)+'_DATA_Clean_Merged.csv'
    group.to_csv(file_name)
    cnt+=1

1  from  144
6  from  144
11  from  144
16  from  144
21  from  144
26  from  144
31  from  144
36  from  144
41  from  144
46  from  144
51  from  144
56  from  144
61  from  144
66  from  144
71  from  144
76  from  144
81  from  144
86  from  144
91  from  144
96  from  144
101  from  144
106  from  144
111  from  144
116  from  144
121  from  144
126  from  144
131  from  144
136  from  144
141  from  144
Wall time: 27min 3s
